In [ ]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import missingno
%matplotlib inline

# Set pandas output display to have one digit for decimal places and limit it to
# printing 15 rows.
pd.options.display.float_format = '{:.2f}'.format
pd.options.display.max_rows = 25

In [ ]:
#reading multiple data source
#Read files
filenames = ['partnership_map.csv', 'touchpoints.csv','trades.csv']
dataframes = [pd.read_csv(f) for f in filenames] 
#call files on the list
pm=dataframes[0]
tp=dataframes[1]
tr=dataframes[2] 

#training and validation
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42, stratify=y)


#Or just do cross validation

#EDA


#rank missing values desc
def missing_df(df):
    missing_df=pd.DataFrame(df.apply(lambda x:sum(x.isnull())/len(df)))
    missing_df.columns=['pct_missing']
    missing_df=missing_df.sort_values(by='pct_missing', ascending=False)
    return missing_df
#dtypes and descriptions after removing missing values
def data_types(df):
  info=df.describe()
  return display(df.dtypes, info)

#drop duplicates
def drop_dup(df):
  df=df.drop_duplicates()
  return df

#reindex
def reindex(df):    
    df=df.reindex(np.random.permutation(df.index))
    return df

#Deleting variables that are missing more than 90% and drop duplicates
  def drop_missing_dup(df):
    drop_missing=df.dropna(thresh=len(df)*0.9, axis='columns')
    drop_missing_dup=drop_missing.drop_duplicates()
    return drop_missing_dup

#seperate number and strings...
def number(self):
    df_number=df.select_dtypes(include='number')
    df_object=df.select_dtypes(include='object')
    return df_number, df_object


#descending sorting by col
def sort(df, col):
    return df.sort_values(by=col, ascending=False)
   
   # get dummy variables

def get_dummies(df_object, df_number):
    for i in range(len(df_object.columns)):
        z=pd.get_dummies(df_object.iloc[:, i], prefix=df_object.columns[i]) #create variables with new names
        df_object=df_object.join(z)
        df_model=df_number.join(df_object) #join both numeric and OHEed cols.
    return df_model.select_dtypes(include='number')

def num_unique(df): #count number of levels for each variable
    return pd.DataFrame(df.apply(lambda column:column.nunique()) )
    
def low_variance(df):
    from sklearn.feature_selection import VarianceThreshold
    def variance_threshold(df, threshold=0.8):
        selector=VarianceThreshold(threshold)
        selector.fit(df)
        return df[df.columns[selector.get_support(indices= True)]]
    return [i for i in df.columns if i not in variance_threshold(df).columns] #return variables that need to be dropped. 

## Visualization

In [ ]:
def histograms_numeric_columns(df, numerical_columns):
    '''
    Takes df, numerical columns as list
    Returns a group of histagrams
    '''
    f = pd.melt(df, value_vars=numerical_columns) 
    g = sns.FacetGrid(f, col='variable',  col_wrap=4, sharex=False, sharey=False)
    g = g.map(sns.distplot, 'value')
    return g


General Data considerations:
1. Binary classifications:
    a. target variable: is it a rare event?
    b. how much money is on the line? which can you tolerate more: false positives or false negatives...
    c. Feature variables: EDA, missing values, outliers (log transformation), visuals, scales, multicollinearity, coded correctly? binning..low        variance (

## Xgboost

In [ ]:
#Baseline logistic Regression model
from sklearn.linear_model import LogisticRegression
log_reg = LogisticRegression(solver='lbfgs')

log_reg.fit(X_train, y_train) #use target1
logreg_predictions1_s = log_reg.predict(X_valid)
precision_score(y_valid, logreg_predictions1_s)


#Baseline Linear Regression model
#retrieve coefficients
regressor = LinearRegression()  
regressor.fit(X_train, y_train) #training the algorithm
regressor.score(X, y)
coeff_df = pd.DataFrame(regressor.coef_, X_train.columns, columns=['Coefficient'])  
coeff_df

#prediction and measure performance 
lineareg_predictions1_s = regressor.predict(X_valid)
mean_squared_error(y_valid, lineareg_predictions1_s)

In [ ]:
#Xgboost Classification
import xgboost as xgb
model1 = xgb.XGBClassifier(silent=False, 
                      scale_pos_weight=1,
                      learning_rate=0.1,  
                      colsample_bytree = 0.6,
                      subsample = 0.8,
                      objective='binary:logistic', 
                      n_estimators=200, 
                      reg_alpha = 0.3,
                      max_depth=6, 
                      gamma=10,
                      seed=42
                    )
eval_set_ns = [(X_train, y_train), (X_valid, y_valid)]


%time model1.fit(X_train, y_train, early_stopping_rounds=5, eval_metric=["logloss", 'error'], eval_set=eval_set_ns, verbose=True)

#plot the training 
epochs = len(results['validation_0']['error'])
x_axis = range(0, epochs)
# plot log loss
fig, ax = pyplot.subplots()
ax.plot(x_axis, results['validation_0']['logloss'], label='Train')
ax.plot(x_axis, results['validation_1']['logloss'], label='Valid')
ax.legend()
pyplot.ylabel('Log Loss')
pyplot.title('XGBoost Log Loss')
pyplot.show()
# plot classification error
fig, ax = pyplot.subplots()
ax.plot(x_axis, results['validation_0']['error'], label='Train')
ax.plot(x_axis, results['validation_1']['error'], label='Valid')
ax.legend()
pyplot.ylabel('Classification Error')
pyplot.title('XGBoost Classification Error')
pyplot.show()

xgb_predict= model1.predict(valid)

from sklearn.metrics import precision_score
precision_score(y_valid, xgb_predict)

from sklearn.metrics import roc_auc_score
roc_auc_score(y_valid, xgb_predict)

In [ ]:
import xgboost as xgb

# regression 
model1 = xgb.XGBRegressor(silent=False, 
                      scale_pos_weight=1,
                      learning_rate=0.01,  
                      colsample_bytree = 0.6,
                      subsample = 0.8,
                       
                      n_estimators=500, 
                      reg_alpha = 0.3,
                      max_depth=6, 
                      gamma=10,
                      seed=42

                    )
eval_set = [(X_train, y_train), (X_valid, y_valid)]

#change evaluation criteria to 'RMSE'
model1.fit(X_train, y_train, early_stopping_rounds=5, eval_metric=["rmse"], eval_set=eval_set, verbose=False)
results =model1.evals_result()


from matplotlib import pyplot
epochs = len(results['validation_0']['rmse'])
x_axis = range(0, epochs)
# plot log loss
fig, ax = pyplot.subplots()
ax.plot(x_axis, results['validation_0']['rmse'], label='Train')
ax.plot(x_axis, results['validation_1']['rmse'], label='Valid')
ax.legend()
pyplot.ylabel('RMSE')
pyplot.title('XGBoost RMSE')
pyplot.show()


#%time model1.fit(X_train, y_train, early_stopping_rounds=5, eval_set=eval_set_ns, verbose=True)
xgb_predict= model1.predict(X_valid)

from sklearn.metrics import mean_squared_error

# #mean_squared_error(y_valid, xgb_predict)

from math import sqrt

rmse = sqrt(mean_squared_error(y_valid, xgb_predict))
print (rmse)


# Cross validation

In [ ]:
def display_scores(scores):
    print("Scores: {0}\nMean: {1:.3f}\nStd: {2:.3f}".format(scores, np.mean(scores), np.std(scores)))

# diabetes = load_diabetes()
# X = diabetes.data
# y = diabetes.target

kfold = KFold(n_splits=10, shuffle=True, random_state=42)

scores = []

for train_index, test_index in kfold.split(X):   
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    xgb_model = xgb.XGBRegressor(
                      objective="reg:squarederror",   #or 'binary:logistic',                
                      seed=42)
    eval_set = [(X_train, y_train), (X_test, y_test)]
    xgb_model.fit(X_train, y_train, early_stopping_rounds=5, eval_metric=["rmse"], #or logloss, 
                  eval_set=eval_set, verbose=False)
    
    y_pred = xgb_model.predict(X_test)
    
    scores.append(mean_squared_error(y_test, y_pred)) #or whatever



    
display_scores(np.sqrt(scores))

## Neural Net

In [ ]:
def construct_feature_columns(input_features):
      return set([tf.feature_column.numeric_column(my_feature)
              for my_feature in input_features])
def my_input_fn(features, targets, batch_size=1, shuffle=True, num_epochs=None):

    # Convert pandas data into a dict of np arrays.
    features = {key:np.array(value) for key,value in dict(features).items()}                                             
 
    # Construct a dataset, and configure batching/repeating.
    ds = Dataset.from_tensor_slices((features,targets)) # warning: 2GB limit
    ds = ds.batch(batch_size).repeat(num_epochs)
    
    # Shuffle the data, if specified.
    if shuffle:
        ds = ds.shuffle(10000)
    
    # Return the next batch of data.
    features, labels = ds.make_one_shot_iterator().get_next()
    return features, labels


def train_nn_regression_model(
    learning_rate,
    steps,
    batch_size,
    hidden_units,
    training_examples,
    training_targets,
    validation_examples,
    validation_targets):
      periods = 10
    steps_per_period = steps / periods
  
  # Create a DNNRegressor object.
  my_optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
  my_optimizer = tf.contrib.estimator.clip_gradients_by_norm(my_optimizer, 5.0)
  dnn_regressor = tf.estimator.DNNRegressor(
      feature_columns=construct_feature_columns(training_examples),
      hidden_units=hidden_units,
      optimizer=my_optimizer,
  )
  
  # Create input functions.
  training_input_fn = lambda: my_input_fn(training_examples, 
                                          training_targets, 
                                          batch_size=batch_size)
  predict_training_input_fn = lambda: my_input_fn(training_examples, 
                                                  training_targets, 
                                                  num_epochs=1, 
                                                  shuffle=False)
  predict_validation_input_fn = lambda: my_input_fn(validation_examples, 
                                                    validation_targets, 
                                                    num_epochs=1, 
                                                    shuffle=False)

  # Train the model, but do so inside a loop so that we can periodically assess
  # loss metrics.
  print("Training model...")
  print("RMSE (on training data):")
  training_rmse = []
  validation_rmse = []
  for period in range (0, periods):
    # Train the model, starting from the prior state.
    dnn_regressor.train(
        input_fn=training_input_fn,
        steps=steps_per_period
    )
    # Take a break and compute predictions.
    training_predictions = dnn_regressor.predict(input_fn=predict_training_input_fn)
    training_predictions = np.array([item['predictions'][0] for item in training_predictions])
    
    validation_predictions = dnn_regressor.predict(input_fn=predict_validation_input_fn)
    validation_predictions = np.array([item['predictions'][0] for item in validation_predictions])
    
    # Compute training and validation loss.
    training_root_mean_squared_error = math.sqrt(
        metrics.mean_squared_error(training_predictions, training_targets))
    validation_root_mean_squared_error = math.sqrt(
        metrics.mean_squared_error(validation_predictions, validation_targets))
    # Occasionally print the current loss.
    print("  period %02d : %0.2f" % (period, training_root_mean_squared_error))
    # Add the loss metrics from this period to our list.
    training_rmse.append(training_root_mean_squared_error)
    validation_rmse.append(validation_root_mean_squared_error)
  print("Model training finished.")

  # Output a graph of loss metrics over periods.
  plt.ylabel("RMSE")
  plt.xlabel("Periods")
  plt.title("Root Mean Squared Error vs. Periods")
  plt.tight_layout()
  plt.plot(training_rmse, label="training")
  plt.plot(validation_rmse, label="validation")
  plt.legend()

  print("Final RMSE (on training data):   %0.2f" % training_root_mean_squared_error)
  print("Final RMSE (on validation data): %0.2f" % validation_root_mean_squared_error)

  return dnn_regressor


dnn_regressor = train_nn_regression_model(
    learning_rate=0.01,
    steps=200,
    batch_size=100,
    hidden_units=[10, 10],
    training_examples=X_train,
    training_targets=y_train,
    validation_examples=X_valid,
    validation_targets=y_valid)